# Liquidity Pool Analysis Workflow

This notebook implements the workflow:
1. Fetch pools based on chains, DEXes, TVL, and volume.
2. Download and save price data to 'data' folder for reuse.
3. Filter pools older than 2 weeks.
4. Compute metrics (volume, fees, ratios, volatility).
5. Filter by metrics to get candidates.
6. Display correlation matrix for candidates.

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from src.clients.geckoterminal import GeckoTerminalClient
from src.constants import Network, NETWORK_CONFIG
from src.utils.analyzer import Analyzer
from datetime import datetime, timedelta
import os
import glob
import numpy as np
import traceback

# Initialize styles
display(HTML('<style>.output {max-height: 500px; overflow-y: auto;}</style>'))

# Define calculation functions
def calculate_volume_and_fees_from_two(base_df: pd.DataFrame, tvl: float, days_interval: int = 7, fee_rate: float = 0.003) -> dict:
    base_df['volume'] = pd.to_numeric(base_df['volume'], errors='coerce')
    if not pd.api.types.is_datetime64_any_dtype(base_df['timestamp']):
        base_df['timestamp'] = pd.to_datetime(base_df['timestamp'], errors='coerce')
    base_df = base_df.dropna(subset=['timestamp', 'volume']).sort_values('timestamp')
    max_time = base_df['timestamp'].max()
    days_interval_ago = max_time - pd.Timedelta(days=days_interval)
    base_interval_df = base_df[base_df['timestamp'] >= days_interval_ago]
    total_volume = base_interval_df['volume'].sum()
    total_fees = total_volume * fee_rate
    fee_to_tvl_ratio = total_fees / tvl if tvl > 0 else None
    volume_to_tvl_ratio = total_volume / tvl if tvl > 0 else None
    return {
        f'{days_interval}days_total_volume': total_volume,
        f'{days_interval}days_fees': total_fees,
        'fee_to_tvl_ratio': fee_to_tvl_ratio,
        'volume_to_tvl_ratio': volume_to_tvl_ratio
    }

def calculate_cross_price_volatility_from_df(cross_df: pd.DataFrame, days_interval: int) -> dict:
    if 'close' not in cross_df.columns:
        if 'price' in cross_df.columns:
            cross_df = cross_df.rename(columns={'price': 'close'})
        else:
            return {}
    if not pd.api.types.is_datetime64_any_dtype(cross_df['timestamp']):
        cross_df['timestamp'] = pd.to_datetime(cross_df['timestamp'], errors='coerce')
    cross_df = cross_df.dropna(subset=['timestamp', 'close']).sort_values('timestamp')
    max_time = cross_df['timestamp'].max()
    start_time = max_time - pd.Timedelta(days=days_interval)
    filtered_df = cross_df[cross_df['timestamp'] >= start_time]
    if filtered_df.empty:
        return {'historical_volatility': np.nan}
    filtered_df['log_return'] = np.log(filtered_df['close'] / filtered_df['close'].shift(1))
    historical_volatility = filtered_df['log_return'].std() * np.sqrt(365 * 24)
    return {'historical_volatility': historical_volatility}

def calculate_pool_metrics(tvl: float, base_df: pd.DataFrame, cross_price_df: pd.DataFrame, days_interval: int = 7, fee_rate: float = 0.003) -> dict:
    volume_metrics = calculate_volume_and_fees_from_two(base_df, tvl, days_interval, fee_rate)
    volatility_metrics = calculate_cross_price_volatility_from_df(cross_price_df, 15)
    return {**volume_metrics, **volatility_metrics}

## Step 1: Fetch Pools

In [2]:
# Define available chains and DEXes
chains = [Network.SOLANA.value, Network.ETHEREUM.value, Network.BSC.value, Network.POLYGONPOS.value,
          Network.ARBITRUM.value, Network.SUI.value, Network.BASE.value]
dexes_by_chain = {
    chain: NETWORK_CONFIG[chain]['dexes'] for chain in chains
}

# Create widgets with multiple selection
chain_dropdown = widgets.SelectMultiple(
    options=chains, 
    description='Chains:', 
    value=[chains[0]],
    rows=5
)
dex_dropdown = widgets.SelectMultiple(
    options=dexes_by_chain[chains[0]], 
    description='DEXes:', 
    value=[],
    rows=5
)

# Fetching widgets
chain_dropdown = widgets.SelectMultiple(options=chains, description='Chains:', value=[chains[0]], rows=5)
dex_dropdown = widgets.SelectMultiple(options=dexes_by_chain[chains[0]], description='DEXes:', value=[], rows=5)
min_tvl = widgets.FloatText(value=10000, description='Min TVL ($):')
max_tvl = widgets.FloatText(value=10000000000.0, description='Max TVL ($):')
min_volume = widgets.FloatText(value=5000, description='Min Volume ($):')
no_pivots = widgets.Checkbox(value=False, description='Exclude Pivot Tokens')
no_stables = widgets.Checkbox(value=True, description='Exclude Stablecoins')
utility_pairs = widgets.Checkbox(value=False, description='Utility Pairs Only')
force_redownload = widgets.Checkbox(value=False, description='Force Re-Download')
fetch_button = widgets.Button(description='Fetch Pools', button_style='success')
output = widgets.Output()

def update_dex_dropdown(change):
    selected_chains = change['new']
    all_dexes = set()
    for chain in selected_chains:
        all_dexes.update(dexes_by_chain.get(chain, []))
    dex_dropdown.options = sorted(all_dexes)
chain_dropdown.observe(update_dex_dropdown, names='value')

display(widgets.VBox([chain_dropdown, dex_dropdown, min_tvl, max_tvl, min_volume, no_pivots, no_stables, utility_pairs, force_redownload, fetch_button, output]))

def fetch_pools(b):
    with output:
        output.clear_output()
        print('Fetching pools...')
        try:
            selected_chains = list(chain_dropdown.value)
            selected_dexes = set(dex_dropdown.value)
            all_pools = []
            for chain in selected_chains:
                chain_dexes = dexes_by_chain.get(chain, [])
                dexes_to_fetch = [dex for dex in chain_dexes if dex in selected_dexes] if selected_dexes else chain_dexes
                for dex_name in dexes_to_fetch:
                    client = GeckoTerminalClient(network=chain, dex=dex_name)
                    pools = client.fetch_liquidity_pools(all_pages=True, min_tvl=min_tvl.value, max_tvl=max_tvl.value, min_volume=min_volume.value, no_pivots=no_pivots.value, no_stables=no_stables.value, utility_pairs=utility_pairs.value)
                    for pool in pools:
                        pool.chain = chain
                        pool.dex = dex_name
                    all_pools.extend(pools)
            
            df = pd.DataFrame([
                {'Chain': pool.chain, 'DEX': pool.dex, 'Pair': f'{pool.token0_symbol}/{pool.token1_symbol}', 'TVL': pool.tvl, 'Volume': pool.volume, 'Address': pool.address}
                for pool in all_pools
            ])
            display(df)
            output_file = f'pools{datetime.now().strftime("%Y%m%d%H%M%S")}.csv'
            df.to_csv(output_file, index=False)
            print(f'Saved to {output_file}')
            update_pool_options_and_data(None)
        except Exception as e:
            print(f'Error: {e}')
            traceback.print_exc()

fetch_button.on_click(fetch_pools)

## Step 2: Download Price Data and Filter by Age

In [7]:
# Function to download data and filter age
def update_pool_options_and_data(change):
    with output:
        print('Downloading data and filtering age...')
        try:
            csv_files = glob.glob('pools_*.csv')
            latest_csv = max(csv_files, key=os.path.getctime)
            df = pd.read_csv(latest_csv)
            
            filtered_pools = []
            for row in df.iterrows():
                chain = row['Chain']
                dex_name = row['DEX']
                address = row['Address']
                pair = row['Pair']
                token0_symbol, token1_symbol = pair.split('/')
                data_folder = os.path.join('data', chain, dex_name, f'{token0_symbol}{token1_symbol}')
                os.makedirs(data_folder, exist_ok=True)
                
                base_usd_path = os.path.join(data_folder, 'base_usd.csv')
                quote_usd_path = os.path.join(data_folder, 'quote_usd.csv')
                cross_path = os.path.join(data_folder, 'cross.csv')
                
                client = GeckoTerminalClient(network=chain, dex=dex_name)
                
                if not force_redownload.value and os.path.exists(base_usd_path) and os.path.exists(quote_usd_path) and os.path.exists(cross_path):
                    cross_df = pd.read_csv(cross_path)
                else:
                    base_usd_bars = client.get_price_bars(address, timeframe='hour', aggregate=1, currency='usd', token='base', limit=1000)
                    quote_usd_bars = client.get_price_bars(address, timeframe='hour', aggregate=1, currency='usd', token='quote', limit=1000)
                    cross_bars = client.get_price_bars(address, timeframe='hour', aggregate=1, currency='token', token='base', limit=1000)
                    if base_usd_bars and quote_usd_bars and cross_bars:
                        base_usd_bars.data.to_csv(base_usd_path, index=False)
                        quote_usd_bars.data.to_csv(quote_usd_path, index=False)
                        cross_bars.data.to_csv(cross_path, index=False)
                        cross_df = cross_bars.data
                    else:
                        continue
                
                # Filter by age
                if len(cross_df) < 336:
                    continue
                cross_df['timestamp'] = pd.to_datetime(cross_df['timestamp'])
                time_span_hours = (cross_df['timestamp'].max() - cross_df['timestamp'].min()).total_seconds() / 3600
                if time_span_hours < 336:
                    continue
                
                filtered_pools.append(row)
            
            filtered_df = pd.DataFrame(filtered_pools)
            display(filtered_df)
            filtered_file = f'filtered_age_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            filtered_df.to_csv(filtered_file, index=False)
            print(f'Saved age-filtered pools to {filtered_file}')
            compute_metrics_button.on_click(lambda x: compute_metrics(x, filtered_file))
        except Exception as e:
            print(f'Error: {e}')
            traceback.print_exc()

## Step 3: Compute Metrics

In [6]:
# Compute metrics button
compute_metrics_button = widgets.Button(description='Compute Metrics', button_style='info')
metrics_output = widgets.Output()
display(compute_metrics_button, metrics_output)

# Compute metrics function
def compute_metrics(b, filtered_file):
    with metrics_output:
        metrics_output.clear_output()
        print('Computing metrics...')
        try:
            df = pd.read_csv(filtered_file)
            metrics_list = []
            for row in df.iterrows():
                chain = row['Chain']
                dex_name = row['DEX']
                address = row['Address']
                tvl = row['TVL']
                pair = row['Pair']
                token0_symbol, token1_symbol = pair.split('/')
                data_folder = os.path.join('data', chain, dex_name, f'{token0_symbol}{token1_symbol}')
                base_usd_path = os.path.join(data_folder, 'base_usd.csv')
                cross_path = os.path.join(data_folder, 'cross.csv')
                base_df = pd.read_csv(base_usd_path)
                cross_df = pd.read_csv(cross_path)
                metrics = calculate_pool_metrics(tvl, base_df, cross_df)
                metrics['Chain'] = chain
                metrics['DEX'] = dex_name
                metrics['Pair'] = pair
                metrics['Address'] = address
                metrics['TVL'] = tvl
                metrics_list.append(metrics)
            
            metrics_df = pd.DataFrame(metrics_list)
            display(metrics_df)
            metrics_file = f'metrics_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            metrics_df.to_csv(metrics_file, index=False)
            print(f'Saved metrics to {metrics_file}')
            filter_metrics_button.on_click(lambda x: filter_metrics(x, metrics_file))
        except Exception as e:
            print(f'Error: {e}')
            traceback.print_exc()

Button(button_style='info', description='Compute Metrics', style=ButtonStyle())

Output()

## Step 4: Filter by Metrics

In [5]:
# Filtering widgets
min_fee_tvl = widgets.FloatText(value=5, description='Min Fee/TVL:')
filter_metrics_button = widgets.Button(description='Filter by Metrics', button_style='info')
filter_output = widgets.Output()
display(min_fee_tvl, filter_metrics_button, filter_output)

# Filter metrics function
def filter_metrics(b, metrics_file):
    with filter_output:
        filter_output.clear_output()
        print('Filtering by metrics...')
        try:
            df = pd.read_csv(metrics_file)
            filtered_df = df[df['fee_to_tvl_ratio'] >= min_fee_tvl.value]
            display(filtered_df)
            candidate_file = f'candidates_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            filtered_df.to_csv(candidate_file, index=False)
            print(f'Saved candidates to {candidate_file}')
            corr_button.on_click(lambda x: compute_correlation(x, candidate_file))
        except Exception as e:
            print(f'Error: {e}')
            traceback.print_exc()

FloatText(value=5.0, description='Min Fee/TVL:')

Button(button_style='info', description='Filter by Metrics', style=ButtonStyle())

Output()

## Step 5: Compute Correlation Matrix

In [4]:
# Correlation button
corr_button = widgets.Button(description='Compute Correlation', button_style='info')
corr_output = widgets.Output()
display(corr_button, corr_output)

# Compute correlation function
def compute_correlation(b, candidate_file):
    with corr_output:
        corr_output.clear_output()
        print('Computing correlation...')
        try:
            df = pd.read_csv(candidate_file)
            data_list = []
            for row in df.iterrows():
                chain = row['Chain']
                dex_name = row['DEX']
                pair = row['Pair']
                token0_symbol, token1_symbol = pair.split('/')
                data_folder = os.path.join('data', chain, dex_name, f'{token0_symbol}{token1_symbol}')
                cross_path = os.path.join(data_folder, 'cross.csv')
                cross_df = pd.read_csv(cross_path)
                data_list.append((pair, cross_df))
            corr_matrix = Analyzer.compute_correlation_matrix_from_dataframes(data_list)
            display(corr_matrix)
            plt.figure(figsize=(10, 8))
            plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
            plt.colorbar()
            plt.xticks(range(len(corr_matrix)), corr_matrix.columns, rotation=90)
            plt.yticks(range(len(corr_matrix)), corr_matrix.index)
            plt.title('Correlation Matrix')
            plt.show()
        except Exception as e:
            print(f'Error: {e}')
            traceback.print_exc()

Button(button_style='info', description='Compute Correlation', style=ButtonStyle())

Output()